Lembremos qual é o nosso projeto: desenvolver, em Python, um
prompt de comando em que se pode inserir frases, formuladas numa
sintaxe próxima da linguagem natural (que definiremos em momento
oportuno), que irão compor uma “base de crenças” de uma pessoa.
Para cada nova inserção *p*, deve-se verificar

1. se *p* é uma tautologia ou contradição; se esse for o caso, não adicionar *p* na base de dados e imprimir “*p* não possui conteúdo
informacional”.
2. se *p* é uma consequência dedutiva de outras sentenças já inseridas;
se esse for o caso, não adicionar *p* na base de dados e imprimir
“informação redundante”;
3. se a adição de *p* faz a base de dados ficar inconsistente; nesse
caso, não adicionar *p* na base de dados e imprimir “informação
conflitante com a base de crenças” (opcional: deixar o usuário
escolher entre desistir de adicionar *p* ou remover outras frases já
inseridas até que a inserção de *p* não torne mais a base de crenças
inconsistente).


### Especificação da sintaxe de entrada de dados
A sintaxe da entrada de dados que escolheremos, em um primeiro
momento, será muito próxima à da linguagem da lógica proposicional; exceto que, ao invés de P, Q, R, etc., (i.e., dos símbolos usados
para frases atômicas) utilizaremos frases do português. A título de
exemplo, as frases ‘ “(a gasolina acabou ∨ o motor fundiu)”, “(a
gasolina não acabou ⊃ o motor fundiu)” e “¬ a gasolina acabou” são
frases bem formadas para a entrada de dados de nosso prompt.

Se a frase começar com uma negação, então ela é o conectivo principal
Se encontrar um conectivo e a contagem de parenteses for 1, então ele é o conectivo principal

Cada conectivo principal vai ser um nó da árvore
As expressões atômicas vão ser os nós filhos do nó principal

Uma expressão atômica é uma letra proposicional

Ao somar 1 para cada parêntese aberto e subtrair 1 para cada parêntese fechado, o resultado deve ser 0

**Tarefa 1**: defina uma função `is_wff(input)` para determinar se
uma frase da lógica proposicional é bem formada ou não.

Para definir se uma frase é bem formada é preciso:

Achar o primeiro conectivo principal
Verificar se o conectivo principal é válido
Separar a expressão pelo conectivo encontrado
Verificar se as expressões separadas são bem formadas



In [31]:
def handle_negation(formula):
    """Check if formula starts with the negation symbol and handle it."""
    negation = ['¬', '~']
    return formula[0] in negation and is_well_formed(formula[1:])


def find_operator_position(formula):
    """Find the position of the '*' operator at the top level."""
    count, index = 0, -1
    for i, char in enumerate(formula):
        if char == '(':
            count += 1
        elif char == ')':
            count -= 1
        elif char in ["∧", "∨", "≡", "⊃", "&", "*"] and count == 1:
            index = i
            return i
    return index


def has_disallowed_chars(formula):
    """Check if formula contains any of the disallowed characters."""
    for char in ['(', ')', '*']:
        if char in formula: return True
    return False


def is_well_formed(formula):
    """Check if a formula is well-formed recursively."""
    # Base cases
    if not formula: return False

    # Handle negation
    if handle_negation(formula): return True

    # Check for balanced parentheses and presence of the '*' operator
    if formula[0] == '(' and formula[-1] == ')':
        operator_position = find_operator_position(formula)
        return operator_position > 0 and is_well_formed(formula[1:operator_position]) and is_well_formed(
            formula[operator_position + 1:-1])

    # Check for any disallowed characters in the formula
    return not has_disallowed_chars(formula)


def is_wff(formula):
    list_op = ["∧", "∨", "≡", "⊃", "&"]
    for op in list_op: formula = formula.replace(op, "*")
    formula = formula.replace(" ", "")
    return is_well_formed(formula)




assert is_wff('((P ∨ (R ∧ ¬S)) ∨ ¬(Q ∧ ¬P))') is True
assert is_wff('¬(P ∧ ¬(¬Q ∨ R))') is True
assert is_wff('((P ∨ (R ∧ ¬S)) ∨ ¬(Q ∧ ¬P))') is True
assert is_wff('(((((R ∧ S) ∨ ¬P) ∨ ¬¬Q) ∨ ¬(S ∧ (¬Q ∨ R))) ∧ ¬(P ∧ ¬(¬Q ∨R)))') is True
assert is_wff('bla') is True
assert is_wff('P & Q') is False
assert is_wff('~(P&Q)') is True
assert is_wff('¬(P ∧ ¬(¬Q ∨ R))') is True
assert is_wff('((P ∨ (R ∧ ¬S)) ∨∨ ¬(Q ∧ ¬P))') is False

**Tarefa 2**: defina uma função `get_tree(input)` que retorna a árvore sintática de uma frase bem formada. Para isso, construa uma
classe Node que representa um nó da árvore. A classe nó deve conter
um campo para guardar uma informação (que será um conectivo ou
uma frase atômica) e uma lista de nós-filho.

In [34]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

    def __repr__(self):
        return f"Node({self.value}, {self.children})"

def get_tree(formula):
    # Se a formula for atômica, retorne um nó com essa formula

    formula = formula.replace(" ", "")

    if is_wff(formula) and len(formula) == 1:
        return Node(formula)

    # Se a formula começa com uma negação, crie um nó para a negação e um nó filho para o resto da formula
    if formula[0] in ['¬', '~']:
        node = Node(formula[0])
        node.children.append(get_tree(formula[1:]))
        return node

    # Encontre a posição do conectivo principal
    operator_position = find_operator_position(formula)

    # Crie um nó para o conectivo principal
    node = Node(formula[operator_position])

    # Divida a formula em subfórmulas e chame a função recursivamente para cada uma
    left_subformula = formula[1:operator_position]
    right_subformula = formula[operator_position + 1:-1]

    node.children.append(get_tree(left_subformula))
    node.children.append(get_tree(right_subformula))

    return node

# Teste inicial
test_formula = "((P ∨ (R ∧ ¬S)) ∨ ¬(Q ∧ ¬P))"
result = get_tree(test_formula)
result

Node(∨, 
[Node(∨, 
[Node(P, 
[]), Node(∧, 
[Node(R, 
[]), Node(¬, 
[Node(S, 
[])])])]), Node(¬, 
[Node(∧, 
[Node(Q, 
[]), Node(¬, 
[Node(P, 
[])])])])])

**Tarefa 3**: implemente a técnica de tableaux semânticos ensinada em
sala de aula.

**Tarefa 4**: defina uma função `is_tautology(formula)` que retorna
`True` se formula é uma tautologia, e False caso contrário. Use, para
isso, a técnica de tableaux semânticos.

**Tarefa 5**: defina uma função `is_contradiction(formula)` que
retorna True se formula é uma contradição, e False caso contrário.
Use, para isso, a técnica de tableaux semânticos

**Tarefa 6**: defina uma função `follows_from(conclusion, premisses)`
que retorna `True` se conclusion é uma consequência de premisses,
e False caso contrário. Use, para isso, a técnica de tableaux semânticos.

**Tarefa 7**: defina uma função `get_sentences_with_atoms(beliefs,
atoms)` que retorna uma lista dos beliefs que contêm ao menos
um átomo proposicional de atoms.

**Tarefa 8**: use todas as funções definidas anteriormente para definir a
função `main` de seu programa.

In [ ]:
if __name__ == '__main__':
    dict = {}
    beliefs = []
    new_belief = ''
    while True:
        new_belief = input('Insira uma nova informação ou a palavra \'sair\' para terminar:')
        if new_belief == 'sair': break
        else:
            orig_new_belief = new_belief

            atomic = get_atomic_sentences(new_belief)
            atoms = []
            for atom in atomic:
                if atom not in dict: dict[atom] = 'p' + (len(dict)+1).__str__()
                new_belief = new_belief.replace(atom, dict[atom])
                atoms.append(dict[atom])
            print("new_belief = ", new_belief)
            #test whether new_belief is a well-formed formula
            if not is_wff(new_belief):
                print(orig_new_belief, ' conduz a uma fórmula mal formada.)
            #test whether new_belief is a tautology or a contradiction
            if is_tautology(new_belief) or is_contradiction(new_belief):
                print(orig_new_belief, ' não possui conteúdo informacional.')
                continue

            premisses = get_sentences_with_atoms(beliefs, atoms)
            if premisses:
                # test whether new_belief is redundant with respect to beliefs
                #print("new_belief = ", new_belief)
                #print("beliefs = ", beliefs)
                if follows_from(new_belief, beliefs):
                    print(orig_new_belief, ' é uma informação redundante em relação à base atual de crenças.')
                    continue
                # test whether new_belief is contradictory with respect to beliefs
                if follows_from('~'+new_belief, beliefs):
                    print(orig_new_belief, ' é uma informação conflitante com a base atual de crenças.')
                    continue

            beliefs.append(new_belief)
            print(orig_new_belief, ' adicionada à base de crenças')